In [1]:
import numpy as np
import pandas as pd
from finta import TA
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from joblib import Parallel, delayed
import joblib
import yfinance as yf

In [2]:
# The trading algorithm will be tested and evaluated over three timelines:

# dcb = Dot Com Bubble
dcb_start = '1997-06-01'
dcb_end = '2002-12-01'

# crsh = 2008 Crash
crsh_start = '2007-06-01'
crsh_end = '2012-12-01'

# cvd = COVID-19
cvd_start = '2020-03-01'
cvd_end = '2022-06-01'

short_window = 4
long_window = 100
initial_capital = 100000.0
share_size = 100
start_date = dcb_start
end_date = dcb_end
stock = 'KO'
# stock = 'MSFT'
# stock = 'M'

In [3]:
# Testing Stocks: 
# S&P 500: Coca-Cola (KO)
# Nasdaq 100: Microsoft (MSFT)
# Russell 2000: Macy's (M)

market_data_df = yf.download(
    'KO, MSFT, M'
)

market_data_df.drop(['Adj Close', 'Volume'], axis='columns', inplace=True)
market_data_df.columns = market_data_df.columns.swaplevel(0, 1)
market_data_df.sort_index(axis=1, inplace=True)
market_data_df.dropna().head()

[*********************100%***********************]  3 of 3 completed


KO                                     M                  \
               Close      High       Low      Open   Close    High     Low   
Date                                                                         
1992-02-05  9.765625  9.921875  9.734375  9.875000  8.6250  9.1250  8.2500   
1992-02-06  9.828125  9.843750  9.765625  9.765625  8.4375  8.6250  8.3750   
1992-02-07  9.765625  9.906250  9.734375  9.828125  7.8125  8.4375  7.8125   
1992-02-10  9.843750  9.859375  9.781250  9.781250  7.5625  7.8750  7.5000   
1992-02-11  9.718750  9.828125  9.687500  9.828125  7.6875  7.6875  7.5625   

                        MSFT                                
              Open     Close      High       Low      Open  
Date                                                        
1992-02-05  9.1250  2.651042  2.677083  2.572917  2.598958  
1992-02-06  8.6250  2.630208  2.687500  2.619792  2.635417  
1992-02-07  8.4375  2.619792  2.656250  2.583333  2.630208  
1992-02-10  7.8750  2.593750  2.625000  2.541667  2.617188  
1992-02-11  7.5625  2.552083  2.598958  2.531250  2.588542

In [4]:
def get_under_over_signals(data=pd.DataFrame):
    
    df = data.drop(columns=['Open', 'Low', 'High'])

    df['Actual Returns'] = df['Close'].pct_change()

    df['Signal'] = 0.0
    df['Signal'] = np.where(
        (df['Actual Returns'] >= 0), 1.0, 0.0
    )

    df = df.drop(columns=['Close', 'Actual Returns'])
    df = df.dropna().sort_index(axis='columns')

    return df

In [5]:
def get_fast_slow_sma(data=pd.DataFrame, short_window=short_window, long_window=long_window):
    # Returns a dataframe with date index, SMA Fast, and SMA Slow

    df = data.drop(columns=['Open', 'Low', 'High'])

    # Generate the fast and slow simple moving averages
    df['SMA Fast'] = (
        df['Close'].rolling(window=short_window).mean()
    )
    df['SMA Slow'] = (
        df['Close'].rolling(window=long_window).mean()
    )

    # Sort the index
    df = df.drop(columns='Close').dropna().sort_index(axis='columns')

    return df

In [6]:
# ohlc_df = pd.read_csv('data/markets_ohlc.csv', header=[0,1], index_col=0)
# ohlc_df = market_data_df.copy()
ohlc_df = market_data_df[stock].copy()
ohlc_df = ohlc_df[cvd_start:cvd_end]

In [7]:
svm_SP500 = joblib.load('models/svm_SP 500.pkl')
svm_NASDAQ100 = joblib.load('models/svm_NASDAQ 100.pkl')
svm_RUSSELL2000 = joblib.load('models/svm_RUSSELL 2000.pkl')  

In [8]:
X = get_fast_slow_sma(ohlc_df)
predicted_signals = svm_NASDAQ100.predict(X)

X_start = X.iloc[0].name
X_end = X.iloc[-1].name

y_test = get_under_over_signals(ohlc_df[X_start:X_end]).values
y_test = np.ravel(y_test)

print(classification_report(y_pred=predicted_signals, y_true=y_test))

              precision    recall  f1-score   support

         0.0       0.50      0.22      0.31       232
         1.0       0.51      0.79      0.62       238

    accuracy                           0.51       470
   macro avg       0.51      0.50      0.46       470
weighted avg       0.51      0.51      0.47       470



/opt/anaconda3/envs/algo/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearSVC was fitted without feature names
  warnings.warn(
